In [1]:
import os

In [2]:
male = []
female = []

for file in os.listdir("./male/"):
    f = open(f"./male/{file}", "r", encoding='utf-8')
    tmp = f.read()
    f.close()
    male.append(tmp)

male_sum = sum(map(lambda x : int(x.replace(",", "")), male))


for file in os.listdir("./female/"):
    f = open(f"./female/{file}", "r", encoding='utf-8')
    tmp = f.read()
    f.close()
    female.append(tmp)

female_sum = sum(map(lambda x : int(x.replace(",", "")), female))

In [3]:
print(male_sum, female_sum)

59417587318 64101458996


In [4]:
import pandas as pd

# '성별'과 '총액' 열을 가진 리스트로 데이터를 만듭니다.
data = [['남자', male_sum], ['여자', female_sum]]

# 데이터프레임 생성 및 열 이름 설정
gender_sum_df = pd.DataFrame(data, columns=['성별', '총액'])

# 결과 출력
print(gender_sum_df)

   성별           총액
0  남자  59417587318
1  여자  64101458996


In [5]:
writer = pd.ExcelWriter('성별_sum.xlsx', engine='xlsxwriter')
# 데이터프레임의 객체를 XlsxWriter Excel 객체로 변환
gender_sum_df.to_excel(writer, sheet_name='Sheet1', index=False)


# xlsxwriter workbook and worksheet 객체 호출
workbook  = writer.book
worksheet = writer.sheets['Sheet1']


# 챠트를 추가
chart = workbook.add_chart({'type': 'column'})


# 챠트에 들어갈 데이터 입력 설정
chart.add_series({
    'values':     '=Sheet1!$B$2:$B$3',
    'categories': '=Sheet1!$A$2:$A$3',
    'gap':        100, # column 사이의 간격
})


# 챠트 axes 설정
chart.set_x_axis({'name': '성별', 'num_font':  {'rotation': 45}})
chart.set_y_axis({'name': '총액', 'major_gridlines': {'visible': True}})


# 범례 설정
chart.set_legend({'position': 'none'})


# 챠트 삽입 위치
worksheet.insert_chart('D2', chart)

header_format = workbook.add_format({
    'size' : 10,
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'align' : 'center',
    'fg_color': '#D7E4BC',
    'border': 1})

# 엑셀의 헤더 부분 Cell 포맷팅 
for col_num, value in enumerate(gender_sum_df.columns.values):
    worksheet.write(0, col_num, value, header_format)

# Cell의 포맷팅 설정 
format1 = workbook.add_format({'num_format': '#,##0.00'})
format2 = workbook.add_format({'num_format':  '#,##0.00'})

# B:B 전체, Cell 크기 20, format 설정은 위의 format1
worksheet.set_column('A:A', 20, format1)

# C:C 전체, Cell 크기 기본값 , format 설정은 위의 format2
worksheet.set_column('B:B', 15, format2)

worksheet.write(0,2, "(단위:원)")
writer.close()

In [6]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import formatdate

#id_, pass_ = '구글이메일', '구글에서 받은 비번'
smtp = smtplib.SMTP_SSL("smtp.gmail.com", 465) 
smtp.login(id_, pass_)


email_message_html = """안녕하세요. <br>

<h1> 오우예 엑셀 </h1>

<p><span style="font-weight:bold">감사합니다.</span> </p>

"""


#email_from = "보내는 이메일"
#email_to = "받는 이메일"
email_date = formatdate(localtime=True)
email_subject = "후어"


msg = MIMEMultipart('mixed')

# 보내는 사람 
msg['From'] = email_from

# 받는 사람 
msg['To'] = email_to

# 날짜
msg['Date'] = email_date

# 제목
msg['Subject'] = email_subject

from email.mime.application import MIMEApplication
from email.mime.base import MIMEBase
from email.encoders import encode_base64
from email.header import Header


# 실제 파일 경로를 open함수로 read 한다.
filename = "./성별_sum.xlsx"
target_file = "./" + filename
with open(target_file, "rb") as attach_file:
    # application, octet-stream은 모든 종류의 파일을 전송할 때 사용하는 타입 
    file_data = MIMEBase("application", "octet-stream")
    file_data.set_payload(attach_file.read())
    encode_base64(file_data)
    # add_header에 filename를 전달해야 첨부된 파일명이 보입니다. 
    file_data.add_header('Content-Disposition', 'attachment', filename=filename)
    msg.attach(file_data)


# 본문 추가 (html형식으로 인코딩 방식은 utf-8)
msg.attach(MIMEText(email_message_html, "html", _charset="utf-8"))
#msg.attach(MIMEText(email_message))

smtp.sendmail(id_, email_to, msg.as_string())


{}